In [1]:
import sqlite3

# Path to your SQLite database file
db_file = r"C:/Users/I745988/Downloads/2023-10-25_cti_data_majd/2023-10-25_cti_data_majd.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Example: List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print all table names
print("Tables in the database:")
for table in tables:
    print(table[0])

# Example: Read data from a specific table (replace 'your_table_name' with an actual table name)
table_name = 'pulses'  # Replace with your actual table name
cursor.execute(f"SELECT COUNT(*) FROM {table_name};")

# Fetch all rows from the query result
rows = cursor.fetchall()

# Print the data
print(f"Data from {table_name}:")
for row in rows:
    print(row)

# Close the cursor and the connection when done
cursor.close()
conn.close()


Tables in the database:
pulses
indicators
ip_location
Data from pulses:
(2966,)


In [5]:
import sqlite3

# Path to your SQLite database file
db_file = r"C:\Users\I745988\Downloads\2023-10-25_cti_data_majd\2023-10-25_cti_data_majd.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Name of the table you want to check
table_name = 'pulses'  # Replace with the actual table name

# Execute PRAGMA command to get the columns of the table
cursor.execute(f"PRAGMA table_info({table_name});")

# Fetch all rows from the query result
columns = cursor.fetchall()

# Print the column details (column_name is the 2nd element in the tuple)
print(f"Columns in the table {table_name}:")
for column in columns:
    print(f"Column name: {column[1]}")

# Close the cursor and connection when done
cursor.close()
conn.close()


Columns in the table pulses:
Column name: id
Column name: name
Column name: description
Column name: author_name
Column name: modified
Column name: created
Column name: public
Column name: adversary
Column name: TLP
Column name: revision
Column name: in_group
Column name: is_subscribing
Column name: malware_family


In [6]:
import sqlite3

# Path to your SQLite database file
db_file = r"C:\Users\I745988\Downloads\2023-10-25_cti_data_majd\2023-10-25_cti_data_majd.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Name of the table you want to check
table_name = 'indicators'  # Replace with the actual table name

# Execute PRAGMA command to get the columns of the table
cursor.execute(f"PRAGMA table_info({table_name});")

# Fetch all rows from the query result
columns = cursor.fetchall()

# Print the column details (column_name is the 2nd element in the tuple)
print(f"Columns in the table {table_name}:")
for column in columns:
    print(f"Column name: {column[1]}")

# Close the cursor and connection when done
cursor.close()
conn.close()


Columns in the table indicators:
Column name: id
Column name: pulse_id
Column name: indicator
Column name: type
Column name: created
Column name: content
Column name: title
Column name: description
Column name: expiration
Column name: is_active


In [7]:
import sqlite3

# Path to your SQLite database file
db_file = r"C:\Users\I745988\Downloads\2023-10-25_cti_data_majd\2023-10-25_cti_data_majd.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Name of the table you want to check
table_name = 'ip_location'  # Replace with the actual table name

# Execute PRAGMA command to get the columns of the table
cursor.execute(f"PRAGMA table_info({table_name});")

# Fetch all rows from the query result
columns = cursor.fetchall()

# Print the column details (column_name is the 2nd element in the tuple)
print(f"Columns in the table {table_name}:")
for column in columns:
    print(f"Column name: {column[1]}")

# Close the cursor and connection when done
cursor.close()
conn.close()


Columns in the table ip_location:
Column name: ip
Column name: cityName
Column name: countryName
Column name: latitude
Column name: longitude


In [ ]:
from kafka import KafkaProducer
from kafka.errors import KafkaError

producer = KafkaProducer(bootstrap_servers='localhost:9092')
future = producer.send('indicators-stream', b'Test message')

try:
    record_metadata = future.get(timeout=10)  # Wait for response to confirm the message
    print(f"Message sent to topic {record_metadata.topic} partition {record_metadata.partition}")
except KafkaError as e:
    print(f"Error sending message: {e}")
finally:
    producer.flush()


In [ ]:
from confluent_kafka import Producer
import json
import time

# Function to initialize Kafka producer
def create_kafka_producer(bootstrap_servers):
    return Producer({'bootstrap.servers': bootstrap_servers})

# Function to send data to Kafka
def send_to_kafka(producer, topic, key, value):
    producer.produce(topic, key=key, value=value)
    producer.flush()  # Make sure to flush after sending

# Function to process and send indicators in batches
def process_and_send_to_kafka(indicators, producer, topic, batch_size=100):
    for i in range(0, len(indicators), batch_size):
        batch = indicators[i:i + batch_size]
        serialized_batch = json.dumps(batch)  # Serialize batch to JSON
        print(f"Sending batch {i // batch_size + 1} to Kafka...")
        send_to_kafka(producer, topic, key=None, value=serialized_batch)

# Kafka Configuration
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'  # Confluent Kafka broker address
KAFKA_TOPIC = 'indicators-stream'  # Correct topic name where messages should be sent

# Create Kafka producer
producer = create_kafka_producer(KAFKA_BOOTSTRAP_SERVERS)

# Example indicator data (for testing)
indicators = [{'id': i, 'value': f'Indicator {i}'} for i in range(500)]  # Simulate 500 indicators

# Process and send indicators to Kafka
CHUNK_SIZE = 500  # Number of indicators per batch
limited_indicators = indicators[:CHUNK_SIZE]
print(f"Processing and sending the first {len(limited_indicators)} indicators to Kafka...")

process_and_send_to_kafka(limited_indicators, producer, KAFKA_TOPIC)


In [ ]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import json

# Kafka Consumer Configuration
conf = {
    'bootstrap.servers': 'localhost:9092',  # Your broker's address
    'group.id': 'test-consumer-group',          # Consumer group ID
    'auto.offset.reset': 'earliest'             # Consume from the earliest message if no offset exists
}

# Initialize Kafka Consumer
consumer = Consumer(conf)

# Subscribe to the topic
consumer.subscribe(['indicators-stream'])

try:
    print("Consuming messages from 'indicators-stream'...")
    while True:
        # Poll for new messages
        msg = consumer.poll(timeout=1.0)  # Wait for up to 1 second for a message
        if msg is None:
            continue  # No message received
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print(f"End of partition reached {msg.topic()} {msg.partition()} offset {msg.offset()}")
            else:
                raise KafkaException(msg.error())
        else:
            # Decode and parse the message
            raw_message = msg.value().decode('utf-8')
            print(f"Raw Message: {raw_message}")
            
            try:
                # Attempt to parse the message as JSON
                parsed_message = json.loads(raw_message)
                print("Parsed Message (JSON):")
                print(json.dumps(parsed_message, indent=4))  # Pretty print JSON
            except json.JSONDecodeError:
                print("Message is not in JSON format.")
except KeyboardInterrupt:
    print("\nConsumption interrupted by user.")
finally:
    print("Closing consumer...")
    consumer.close()


In [3]:
from confluent_kafka import Producer
import sqlite3
import json

# SQLite Configuration
DB_FILE = r"C:/Users/I745988/Downloads/2023-10-25_cti_data_majd/2023-10-25_cti_data_majd.db"
TABLES = ['pulses', 'indicators', 'ip_location']  # List of table names

# Kafka Configuration
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'
KAFKA_TOPIC = 'data-stream'

# Initialize Kafka Producer
def create_kafka_producer(bootstrap_servers):
    return Producer({'bootstrap.servers': bootstrap_servers})

# Send data to Kafka
def send_to_kafka(producer, topic, key, value):
    producer.produce(topic, key=key, value=value)
    producer.flush()

# Display and ingest data from SQLite
def display_and_ingest_data(db_file, tables, producer, kafka_topic, row_limit=10):
    # Connect to SQLite
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Loop through each table
    for table_name in tables:
        print(f"\nFetching first {row_limit} rows from table '{table_name}'...")
        cursor.execute(f"SELECT * FROM {table_name} LIMIT {row_limit}")
        rows = cursor.fetchall()

        # Get column names for better display and JSON serialization
        column_names = [description[0] for description in cursor.description]
        
        # Display the rows
        print(f"Columns in '{table_name}': {column_names}")
        for row in rows:
            print(dict(zip(column_names, row)))

        # Serialize and produce each row to Kafka
        print(f"\nProducing rows from '{table_name}' to Kafka...")
        for row in rows:
            serialized_row = json.dumps(dict(zip(column_names, row)))  # Convert to JSON
            send_to_kafka(producer, kafka_topic, key=None, value=serialized_row)

    print("\nData ingestion and production completed.")
    cursor.close()
    conn.close()

# Main Producer Logic
producer = create_kafka_producer(KAFKA_BOOTSTRAP_SERVERS)
display_and_ingest_data(DB_FILE, TABLES, producer, KAFKA_TOPIC)



Fetching first 10 rows from table 'pulses'...
Columns in 'pulses': ['id', 'name', 'description', 'author_name', 'modified', 'created', 'public', 'adversary', 'TLP', 'revision', 'in_group', 'is_subscribing', 'malware_family']
{'id': '645c9a033dd0e4e34e6cba8f', 'name': 'Malware Bazaar 7', 'description': '', 'author_name': 'LoveAndren', 'modified': '2023-05-11T07:32:19.907000', 'created': '2023-05-11T07:32:19.907000', 'public': 1, 'adversary': '', 'TLP': 'white', 'revision': 1, 'in_group': 0, 'is_subscribing': None, 'malware_family': '%23Exploit:NtQueryIntervalProfile'}
{'id': '63ac057b2644355f3788973e', 'name': 'Cyble - New Ransomware Strains Emerging from Leaked Conti’s Source Code', 'description': 'A look at some of the latest ransomware strains created based on the leaked source code of Conti ransomware, which was discovered in 2021 and is being investigated by Cyble Research and Intelligence Labs (CRIL).', 'author_name': 'CyberHunter_NL', 'modified': '2022-12-28T08:59:38.883000', 'c

In [4]:
import sqlite3
import json

# SQLite Configuration
DB_FILE = r"C:/Users/I745988/Downloads/2023-10-25_cti_data_majd/2023-10-25_cti_data_majd.db"
ROW_LIMIT = 10  # Limit the number of rows fetched

# Perform the join operation between 'indicators' and 'ip_location'
def fetch_joined_data(db_file, row_limit):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    join_query = """
    SELECT 
        ind.id AS indicator_id,
        ind.pulse_id,
        ind.indicator,
        ind.type,
        ind.created,
        ind.title,
        ind.description,
        loc.cityName,
        loc.countryName,
        loc.latitude,
        loc.longitude
    FROM 
        indicators AS ind
    INNER JOIN 
        ip_location AS loc
    ON 
        ind.indicator = loc.ip
    LIMIT ?;
    """
    cursor.execute(join_query, (row_limit,))
    joined_rows = cursor.fetchall()

    # Get column names for joined data
    joined_columns = [description[0] for description in cursor.description]

    # Convert rows to dictionary for easier usage
    joined_data = [dict(zip(joined_columns, row)) for row in joined_rows]

    cursor.close()
    conn.close()

    return joined_columns, joined_data

# Fetch joined data
joined_columns, joined_data = fetch_joined_data(DB_FILE, ROW_LIMIT)

# Display the joined data
print("\nJoined Data Columns:")
print(joined_columns)

print("\nJoined Data Rows:")
for row in joined_data:
    print(row)



Joined Data Columns:
['indicator_id', 'pulse_id', 'indicator', 'type', 'created', 'title', 'description', 'cityName', 'countryName', 'latitude', 'longitude']

Joined Data Rows:
{'indicator_id': 2233484595, 'pulse_id': '6518f9615a88e0f1e325bde4', 'indicator': '148.251.234.93', 'type': 'IPv4', 'created': '2023-10-01T04:45:24', 'title': '', 'description': 'CC=DE ASN=AS24940 Hetzner Online GmbH', 'cityName': 'Berlin', 'countryName': 'Germany', 'latitude': '52.524525', 'longitude': '13.410037'}
{'indicator_id': 3762239076, 'pulse_id': '6518f9615a88e0f1e325bde4', 'indicator': '148.251.162.83', 'type': 'IPv4', 'created': '2023-10-01T04:45:24', 'title': '', 'description': 'CC=DE ASN=AS24940 Hetzner Online GmbH', 'cityName': 'Berlin', 'countryName': 'Germany', 'latitude': '52.524525', 'longitude': '13.410037'}
{'indicator_id': 3748688316, 'pulse_id': '6518f9615a88e0f1e325bde4', 'indicator': '148.251.189.114', 'type': 'IPv4', 'created': '2023-10-01T04:45:24', 'title': '', 'description': 'CC=DE 

In [2]:
from confluent_kafka import Producer
import sqlite3
import json
import time

# SQLite Configuration
DB_FILE = r"C:/Users/I745988/Downloads/2023-10-25_cti_data_majd/2023-10-25_cti_data_majd.db"
TABLE_NAME = 'indicators'  # Replace with your table name

# Kafka Configuration
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'
KAFKA_TOPIC = 'indicators-stream'

# Initialize Kafka Producer
def create_kafka_producer(bootstrap_servers):
    return Producer({'bootstrap.servers': bootstrap_servers})

# Send data to Kafka
def send_to_kafka(producer, topic, key, value):
    producer.produce(topic, key=key, value=value)
    producer.flush()

# Ingest data from SQLite and produce to Kafka
def ingest_and_produce(db_file, table_name, producer, kafka_topic, batch_size=100):
    # Connect to SQLite
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Fetch data from the specified table
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Get column names for JSON serialization
    column_names = [description[0] for description in cursor.description]

    # Process and produce data in batches
    for i in range(0, len(rows), batch_size):
        batch = rows[i:i + batch_size]
        serialized_batch = [
            dict(zip(column_names, row)) for row in batch
        ]  # Convert to JSON-serializable format
        serialized_batch_json = json.dumps(serialized_batch)

        print(f"Producing batch {i // batch_size + 1} to Kafka...")
        send_to_kafka(producer, kafka_topic, key=None, value=serialized_batch_json)
        time.sleep(1)  # Optional: Add a delay to control throughput

    print("Data ingestion and production completed.")
    cursor.close()
    conn.close()

# Main Producer Logic
producer = create_kafka_producer(KAFKA_BOOTSTRAP_SERVERS)
ingest_and_produce(DB_FILE, TABLE_NAME, producer, KAFKA_TOPIC)


Producing batch 1 to Kafka...
Producing batch 2 to Kafka...
Producing batch 3 to Kafka...
Producing batch 4 to Kafka...
Producing batch 5 to Kafka...
Producing batch 6 to Kafka...
Producing batch 7 to Kafka...
Producing batch 8 to Kafka...
Producing batch 9 to Kafka...
Producing batch 10 to Kafka...
Producing batch 11 to Kafka...
Producing batch 12 to Kafka...
Producing batch 13 to Kafka...
Producing batch 14 to Kafka...
Producing batch 15 to Kafka...
Producing batch 16 to Kafka...
Producing batch 17 to Kafka...
Producing batch 18 to Kafka...
Producing batch 19 to Kafka...
Producing batch 20 to Kafka...
Producing batch 21 to Kafka...
Producing batch 22 to Kafka...
Producing batch 23 to Kafka...
Producing batch 24 to Kafka...
Producing batch 25 to Kafka...
Producing batch 26 to Kafka...
Producing batch 27 to Kafka...
Producing batch 28 to Kafka...
Producing batch 29 to Kafka...
Producing batch 30 to Kafka...
Producing batch 31 to Kafka...
Producing batch 32 to Kafka...
Producing batch 3

KeyboardInterrupt: 